<a href="https://colab.research.google.com/github/BharathMenon/API_testing_Cyber-o-day_hackathon/blob/main/Corn_Maize_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle


In [ ]:
! mkdir ~/.kaggle



In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#! kaggle competitions download <name-of-competition> - command to download competition dataset
! kaggle datasets download smaranjitghose/corn-or-maize-leaf-disease-dataset # command to download normal dataset



 94% 151M/161M [00:01<00:00, 95.1MB/s]
100% 161M/161M [00:01<00:00, 114MB/s] 


In [ ]:
import tensorflow as tf
from tensorflow.keras import models,layers
#import matplotlib.pyplot as plt
import numpy as np
import shutil


In [ ]:
!unzip corn-or-maize-leaf-disease-dataset.zip

Archive:  corn-or-maize-leaf-disease-dataset.zip
  inflating: data/Blight/Corn_Blight (1).jpeg  
  inflating: data/Blight/Corn_Blight (1).jpg  
  inflating: data/Blight/Corn_Blight (10).jpg  
  inflating: data/Blight/Corn_Blight (100).jpg  
  inflating: data/Blight/Corn_Blight (1000).JPG  
  inflating: data/Blight/Corn_Blight (1001).JPG  
  inflating: data/Blight/Corn_Blight (1002).JPG  
  inflating: data/Blight/Corn_Blight (1003).JPG  
  inflating: data/Blight/Corn_Blight (1004).JPG  
  inflating: data/Blight/Corn_Blight (1005).JPG  
  inflating: data/Blight/Corn_Blight (1006).JPG  
  inflating: data/Blight/Corn_Blight (1007).JPG  
  inflating: data/Blight/Corn_Blight (1008).JPG  
  inflating: data/Blight/Corn_Blight (1009).JPG  
  inflating: data/Blight/Corn_Blight (101).jpg  
  inflating: data/Blight/Corn_Blight (1010).JPG  
  inflating: data/Blight/Corn_Blight (1011).jpg  
  inflating: data/Blight/Corn_Blight (1012).JPG  
  inflating: data/Blight/Corn_Blight (1013).JPG  
  inflatin

In [ ]:

# shutil.rmtree('/content/PlantVillage/Potato___healthy')
# shutil.rmtree('/content/PlantVillage/Potato___Late_blight')
#shutil.rmtree('/content/PlantVillage/Potato___Late_blight')
# shutil.rmtree('/content/PlantVillage/Potato___Early_blight')
# shutil.rmtree('/content/PlantVillage/Pepper__bell___healthy')
shutil.rmtree('/content/PlantVillage')

In [ ]:
class_names = dataset.class_names
class_names

['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']

In [ ]:
img_size = 256
b_size = 32
dataset=tf.keras.preprocessing.image_dataset_from_directory(
"/content/data",
shuffle=True, #every time dataset is accesed all images are shuffled
 image_size = (img_size,img_size),
    batch_size=128 #size of every element in dataset
)

Found 4188 files belonging to 4 classes.


In [ ]:
plt.figure(figsize=(15,15)) #Used to increase figure size
for image_batch,label_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.axis("off")
        plt.title(class_names[label_batch[i]])
#         print(image_batch.shape)
#         print(label_batch.numpy().shape)

NameError: ignored

In [ ]:
def get_dataset_partitions_tf(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    ds_size = len(ds)
    if shuffle:
        ds.shuffle(shuffle_size,seed=7)
    train_size=int(ds_size*train_split)
    test_size = int(ds_size*test_split)
    val_size = int(ds_size*val_split)
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds,val_ds,test_ds

In [ ]:
train_ds,val_ds,test_ds = get_dataset_partitions_tf(dataset)
len(val_ds)

3

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE) # cache() saves all data read in one epoch so that in future epochs it won't have to read
#the data once again, hence optimising process
#prefetch() loads next batch of data while current one is running and hence optimises time.
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(img_size,img_size),
    layers.experimental.preprocessing.Rescaling(1.0/255),

])
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
n_classes = 4
input_shape = (b_size,img_size,img_size,3)
model = models.Sequential([
    #layers.InputLayer(input_shape=input_shape),
    resize_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
   layers.Conv2D(64,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    #layers.Dense(80,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax') #Softmax normalises probability of classes.
])
model.build(input_shape=input_shape)

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
history =  model.fit(train_ds,epochs=80,batch_size=b_size,verbose=1,validation_data=val_ds)

Epoch 1/80
26/26 [==============================] - 525s 20s/step - loss: 1.0212 - accuracy: 0.5472 - val_loss: 0.6298 - val_accuracy: 0.7214
Epoch 2/80
26/26 [==============================] - 487s 19s/step - loss: 0.5253 - accuracy: 0.7831 - val_loss: 0.4566 - val_accuracy: 0.8099
Epoch 3/80
26/26 [==============================] - 486s 19s/step - loss: 0.3958 - accuracy: 0.8308 - val_loss: 0.4320 - val_accuracy: 0.8281
Epoch 4/80
26/26 [==============================] - 490s 19s/step - loss: 0.3734 - accuracy: 0.8380 - val_loss: 0.3920 - val_accuracy: 0.8411
Epoch 5/80
26/26 [==============================] - 487s 19s/step - loss: 0.3553 - accuracy: 0.8477 - val_loss: 0.3825 - val_accuracy: 0.8333
Epoch 6/80
26/26 [==============================] - 489s 19s/step - loss: 0.3549 - accuracy: 0.8489 - val_loss: 0.3655 - val_accuracy: 0.8464
Epoch 7/80
26/26 [==============================] - 493s 19s/step - loss: 0.3720 - accuracy: 0.8447 - val_loss: 0.3724 - val_accuracy: 0.8516
Epoch 

In [ ]:
model.evaluate(test_ds)

4/4 [==============================] - 0s 86ms/step - loss: 0.1726 - accuracy: 0.9328


[0.17259004712104797, 0.9327731132507324]